In [1]:
import sys
import cv2
import numpy as np

In [41]:
left_img = cv2.imread("left.jpeg")
right_img = cv2.imread("right.jpeg")


cv2.imshow("left_image",left_img)
cv2.imshow("right_image",right_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [57]:
orb = cv2.ORB_create(nfeatures=1000)

kpleft, dcleft = orb.detectAndCompute(left_img, None)
kpright, dcright = orb.detectAndCompute(right_img, None)

In [58]:
cv2.imshow("Left keypoints",cv2.drawKeypoints(left_img, kpleft, None, (255, 0, 0)))
cv2.waitKey(0)
cv2.destroyAllWindows()
           

    
           

In [59]:
cv2.imshow("right keypoints",cv2.drawKeypoints(right_img, kpright, None, (255, 0, 0)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
brutef = cv2.BFMatcher_create(cv2.NORM_HAMMING)


matches = brutef.knnMatch(dcleft, dcright,k=2)

In [74]:
print("Descriptor for the first left keypoint: ")
print(dcleft[0])

Descriptor for the first left keypoint: 
[139  18 116 247  99  61 184  88 181  90 149 183 253  93 191 111 255 145
 223  39 164  71 110 194  87 219 114 143  54 252  37 247]


In [75]:
print("Descriptor for the first right keypoint: ")
print(dcright[0])

Descriptor for the first right keypoint: 
[  2  80   8 155 109 137 177  76  30  29  84 163 172 223 107  73 202  25
 163 193  66  22  66 241  68  17 250 171  20  28  29 155]


In [76]:
def draw_matches(left_img, kpleft, right_img, kpright, all_matches):
    left_row, left_col = left_img.shape[:2]
    right_row, right_col = right_img.shape[:2]

  
    match_img = np.zeros((max([left_row,  right_row]), left_col+right_col, 3), dtype='uint8')
    match_img[:left_row, :left_col, :] = np.dstack([left_img, left_img, left_img])
    match_img[: right_row, left_col:left_col+right_col, :] = np.dstack([right_img, right_img, right_img])

  
    for m in all_matches:
        left_img_index = m.queryIdx
        right_img_index = m.trainIdx
        (xleft, yleft) = kpleft[left_img_index].pt
        (xright, yright) = kpright[right_img_index].pt

    
        cv2.circle(match_img, (int(xleft),int(yleft)), 4, (0, 255, 255), 1)
        cv2.circle(match_img, (int(xright)+left_col,int(yright)), 4, (0, 255, 255), 1)

    
        cv2.line(match_img, (int(xleft),int(yleft)), (int(xright)+left_col,int(yright)), (0, 255, 255), 1)
    
    return match_img

In [77]:
all_matches = []
for m, n in matches:
    all_matches.append(m)

match_img = draw_matches(left_img_gray, kpleft, right_img_gray, kpright, all_matches[:30])
cv2.imshow("matches",match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [78]:
good_matches = []
threshold=0.6
for m, n in matches:
    if m.distance < threshold * n.distance:
        good_matches.append(m)

In [79]:
cv2.imshow("key",cv2.drawKeypoints(left_img, [kpleft[m.queryIdx] for m in good_matches], None, (255, 0, 255)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [80]:
cv2.imshow("key",cv2.drawKeypoints(right_img, [kpright[m.trainIdx] for m in good_matches], None, (255, 0, 255)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [81]:
match_img = draw_matches(left_img_gray, kpleft, right_img_gray, kpright, good_matches)
cv2.imshow("good matches",match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [82]:
def matchimages(left_img, right_img, homography):

    left_row, left_col = left_img.shape[:2]
    right_row, right_col = right_img.shape[:2]
    default=0
    size=-1,1,2

    points1 = np.float32([[default,default], [default, left_row],[left_col, left_row], [left_col, default]]).reshape(size)
    temp_points = np.float32([[default,default], [default,right_row], [right_col,right_row], [right_col,default]]).reshape(size)

 
 
    points2 = cv2.perspectiveTransform(temp_points, homography)

    points = np.concatenate((points1,points2), axis=0)

    [minx, miny] = np.int32(points.min(axis=0).ravel() - 0.5)
    [maxx, maxy] = np.int32(points.max(axis=0).ravel() + 0.5)
  
    distance = [-minx,-miny]
  
    homography_translation = np.array([[1, default, distance[default]], [default, 1, distance[1]], [default, default, 1]])

    result = cv2.warpPerspective(right_img, homography_translation.dot(homography), (maxx-minx, maxy-miny))
    result[distance[1]:left_row+distance[1], distance[default]:left_col+distance[default]] = left_img

    return result

In [84]:
matches = 15

if len(good_matches) > matches:
    
    left_kp_matchcontrol=[ kpleft[match.queryIdx].pt for match in good_matches]
    right_kp_matchcontrol=[ kpright[match.trainIdx].pt for match in good_matches]
    size=-1,1,2
    source = np.float32(left_kp_matchcontrol).reshape(size)
    destination = np.float32(right_kp_matchcontrol).reshape(size)

    
    homo, _ = cv2.findHomography(source, destination, cv2.RANSAC,3.0)
    
    result = matchimages(right_img, left_img, homo)

    cv2.imshow("result",result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()